In [75]:
#imports
import k3d
import json
import os
import networkx as nx
import numpy as np
import random


In [76]:
#functions

def rgb_to_hex(r,g,b):
    # Convert to a hexadecimal string
    hex_color = f'{r:02x}{g:02x}{b:02x}'
    # Convert the hexadecimal string to an integer in base-16
    color_int = int(hex_color, 16)
    return color_int
    
def euclidean_distance(node1, node2):
    """
    Calculate the Euclidean distance between two nodes.

    Parameters:
    node1, node2 (dict): Nodes with 'pos' key containing x, y, z coordinates.

    Returns:
    float: Euclidean distance between node1 and node2.
    """
    pos1 = np.array(node1['pos'])
    pos2 = np.array(node2['pos'])
    return np.linalg.norm(pos1 - pos2)


def add_node_to_graph(graph, node):
    """
    Add a node with attributes to the graph.

    Parameters:
    graph (nx.DiGraph): The graph to which the node will be added.
    node (dict): Node data containing sampleNumber, pos, radius, structure_id, and allen_id.
    """
    graph.add_node(
        node['sampleNumber'], 
        pos=(node['x'], node['y'], node['z']), 
        radius=node['radius'], 
        structure_id=node['structureIdentifier'],
        allen_id=node['allenId']
    )
    
def json_to_digraph(file_path):
    """
    Load a neuronal reconstruction from a JSON file into a NetworkX graph.

    The JSON file contains neuronal data with additional brain region information for each node.
    The graph will be a simple, directed, rooted tree with no cycles or nodes with more than one parent.

    Parameters:
    file_path (str): Path to the JSON file containing neuronal reconstruction data.

    Returns:
    nx.DiGraph: A directed graph representing the neuronal tree.
    """
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
    except IOError as e:
        print(f"Error opening file: {e}")
        return None
   
    neuron_data = data['neuron'] if 'neuron' in data else data['neurons'][0]

    axon_graph, dendrite_graph = nx.DiGraph(), nx.DiGraph()
    
    for structure, graph in [('dendrite', dendrite_graph), ('axon', axon_graph)]:
    
        if structure not in neuron_data:
            print(f"Missing structure {structure} for {file_path}")
            continue
        for node in sorted(neuron_data[structure], key=lambda x: x['sampleNumber']):
            
            add_node_to_graph(graph, node)
            
            if node['parentNumber'] != -1:
                
                add_edge_to_graph(graph, node['parentNumber'], node['sampleNumber'])
            
    axon_graph.remove_node(1)  # Remove duplicate soma node from axon graph
    
    # Merge graphs and rename axon nodes to avoid key collisions
    first_axon_label = max(dendrite_graph.nodes()) + 1 if dendrite_graph.nodes() else 1
    joined_graph = nx.union(
        dendrite_graph, 
        nx.convert_node_labels_to_integers(
            axon_graph, 
            first_label=first_axon_label
        )
    )
    
    roots = [n for n in joined_graph if joined_graph.in_degree(n) == 0]
    # Link the dendrite to the axon
    if len(roots) == 2:
        add_edge_to_graph(joined_graph, roots[0], roots[1])

    return joined_graph


def add_edge_to_graph(graph, parent, child):
    """
    Add an edge between parent and child nodes in the graph, with weight as Euclidean distance.

    Parameters:
    graph (nx.DiGraph): The graph to which the edge will be added.
    parent, child (int): The sampleNumbers of the parent and child nodes.
    """
    graph.add_edge(
        parent, 
        child, 
        weight=euclidean_distance(
            graph.nodes()[parent],
            graph.nodes()[child]
        )
    )

def load_graph(file_path):
    """
    Load a single graph from a JSON file.

    Parameters:
    file_path (str): Path to the JSON file.

    Returns:
    nx.Graph: A graph loaded from the JSON file, or None if there's an error.
    """
    try:
        graph = json_to_digraph(file_path)
        return graph
    except Exception as e:
        print(f"Error loading {os.path.basename(file_path)}: {e}")
        return None
        
def graph_to_lines(g,color=None):
    # Extract vertex positions
    g_verts = np.array([g.nodes[n]['pos'] for n in sorted(g.nodes())])
    # Pairs of indices into the vertex array are edges
    # Node keys start at 1
    g_inds = np.array([[u-1, v-1] for u, v in g.edges()])
    if color is None:
        color=random_color_hex()

    
    g_lines = k3d.factory.lines(g_verts, g_inds, indices_type='segment', color=color, width=1, shader='simple')
    return g_lines



def plot_graphs(graphs, plot, color = None):
    for i, g in enumerate(graphs):
        g_lines = graph_to_lines(g,color)
        plot += g_lines

def random_color_hex():
    red = random.randint(0, 255)
    green = random.randint(0, 255)
    blue = random.randint(0, 255)
    return rgb_to_hex(red, green, blue)

In [79]:
#select file and plot
filepath = "/data/neuron_tracings/exaSPIM_653158_2023-06-01_20-41-38_reconstructions/Complete_annotated/N006-653158-consensus.json"
g = load_graph(filepath)
plot = k3d.plot()
color = rgb_to_hex(255,0,0) #red
plot_graphs([g], plot,color)
plot.display()

Output()